In [2]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)

In [14]:
df_egresos = pd.read_csv("../data/processed/df_procesada.csv")

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_52842/666491669.py:1: DtypeWarning: Columns (18,80,93,94,95,97,99,100,101,144,152,153,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df_egresos = pd.read_csv("../data/processed/df_procesada.csv")


In [15]:
df_pabellon = pd.read_csv("../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";")